## Question 1

In [1]:
# Importation des librairies de base

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from scipy.stats import gaussian_kde    
from scipy.stats import probplot
from scipy.stats import ttest_rel

import wrds
from scipy import stats
from scipy.stats import skew, kurtosis
import warnings

### a) Extraction des données WRDS pour 50 sociétés du S&P 500

L'article utilise des données mensuelles. Nous allons extraire:
- Prix mensuels des actions (CRSP)
- Bénéfice par action - EPS (IBES)
- Calculer le ratio P/E glissant (trailing P/E)

In [2]:
# Connexion à WRDS
db = wrds.Connection(wrds_username='vince1209')

Loading library list...
Done


In [3]:
# Définir la période d'analyse (ajustez selon l'article)
# L'article utilise typiquement des données de 1985 à 2015 environ
start_date = '2003-01-01'
end_date = '2023-12-31'

n_compagnies = 50
np.random.seed(42)

In [4]:
# Étape 1: Récupérer la liste des constituants du S&P 500 avec leurs noms et tickers

# Choisir le type de sélection
current_sp500_only = False  # False = toutes les sociétés qui ont été dans le S&P 500 entre 2003 et 2023

# Requête modifiée pour capturer toutes les sociétés qui ont été dans le S&P 500 
# à n'importe quel moment pendant la période 2003-2023
query_sp500 = """
SELECT DISTINCT sp.permno, n.comnam, n.ticker
FROM crsp.dsp500list sp
JOIN crsp.msenames n ON sp.permno = n.permno
WHERE sp.start <= '{end_date}'
  AND sp.ending >= '{start_date}'
  AND n.namedt <= '{end_date}'
  AND n.nameendt >= '{start_date}'
ORDER BY sp.permno
""".format(start_date=start_date, end_date=end_date)

print(f"Récupération de toutes les sociétés qui ont été dans le S&P 500 entre {start_date} et {end_date}...")
sp500_data = db.raw_sql(query_sp500)

print(f"Nombre total de lignes récupérées: {len(sp500_data)}")
print(f"Nombre de PERMNO uniques: {sp500_data['permno'].nunique()}")
print(f"Nombre de TICKER uniques: {sp500_data['ticker'].nunique()}")


# Garder seulement les permno uniques (le ticker le plus récent)
sp500_constituents = sp500_data.drop_duplicates(subset=['permno'], keep='last')

print(f"\nNombre total de sociétés S&P 500 trouvées: {len(sp500_constituents)}")
print("\nPremières lignes:")
print(sp500_constituents)


Récupération de toutes les sociétés qui ont été dans le S&P 500 entre 2003-01-01 et 2023-12-31...
Nombre total de lignes récupérées: 1344
Nombre de PERMNO uniques: 967
Nombre de TICKER uniques: 1092

Nombre total de sociétés S&P 500 trouvées: 967

Premières lignes:
      permno                    comnam ticker
1      10078      SUN MICROSYSTEMS INC   SUNW
2      10104               ORACLE CORP   ORCL
3      10107            MICROSOFT CORP   MSFT
4      10108  SUNGARD DATA SYSTEMS INC    SDS
5      10137      ALLEGHENY ENERGY INC    AYE
...      ...                       ...    ...
1337   93159               VALARIS PLC    VAL
1338   93246      GENERAC HOLDINGS INC   GNRC
1339   93422       Q E P RESOURCES INC    QEP
1341   93429      C B O E HOLDINGS INC   CBOE
1343   93436          TESLA MOTORS INC   TSLA

[967 rows x 3 columns]


In [5]:
# Étape 2: Sélectionner aléatoirement 50 sociétés avec données disponibles entre 2003 et 2023
# Filtrer pour avoir uniquement des sociétés avec des données valides
sp500_valid = sp500_constituents[sp500_constituents['permno'].notna()].copy()

print(f"Nombre de sociétés candidates: {len(sp500_valid)}")
print("Vérification de la disponibilité des données pour chaque société...")

# Récupérer les PERMNOs candidats
candidate_permnos = sp500_valid['permno'].tolist()
permnos_str_temp = ','.join(map(str, candidate_permnos))

# Requête pour compter le nombre d'observations mensuelles par société
query_data_availability = """
SELECT 
    permno,
    COUNT(*) as num_observations,
    MIN(date) as first_date,
    MAX(date) as last_date
FROM crsp.msf
WHERE permno IN ({permnos})
  AND date >= '{start_date}'
  AND date <= '{end_date}'
  AND prc IS NOT NULL
GROUP BY permno
ORDER BY num_observations DESC
""".format(permnos=permnos_str_temp, start_date=start_date, end_date=end_date)

data_availability = db.raw_sql(query_data_availability)

print(f"\nSociétés avec des données: {len(data_availability)}")

# Fusionner avec sp500_valid pour garder les informations complètes
sp500_with_data = sp500_valid.merge(
    data_availability[['permno', 'num_observations', 'first_date', 'last_date']], 
    on='permno', 
    how='inner'
)

# Sélection aléatoire de 50 sociétés
n_companies = 50
if len(sp500_with_data) >= n_companies:
    selected_companies = sp500_with_data.sample(n=n_companies, random_state=42)
    print(f"\n✓ {n_companies} sociétés sélectionnées aléatoirement")
else:
    print(f"\n⚠ Attention: Seulement {len(sp500_with_data)} sociétés disponibles")
    selected_companies = sp500_with_data

# Récupérer les PERMNOs pour les requêtes futures
permnos = selected_companies['permno'].tolist()
permnos_str = ','.join(map(str, permnos))

# Reset index et trier par nombre d'observations
selected_companies = selected_companies.sort_values('num_observations', ascending=False).reset_index(drop=True)

print(f"\n{'='*70}")
print("SOCIÉTÉS SÉLECTIONNÉES")
print(f"{'='*70}")
print(f"\nNombre de sociétés: {len(selected_companies)}")
print(f"\nStatistiques de couverture:")
print(f"  - Observations moyennes par société: {selected_companies['num_observations'].mean():.0f}")
print(f"  - Min: {selected_companies['num_observations'].min()}")
print(f"  - Max: {selected_companies['num_observations'].max()}")

print(f"\n{'='*70}")
print("LISTE DES SOCIÉTÉS")
print(f"{'='*70}")
print(selected_companies[['permno', 'comnam', 'ticker', 'num_observations', 'first_date', 'last_date']].to_string())


Nombre de sociétés candidates: 967
Vérification de la disponibilité des données pour chaque société...

Sociétés avec des données: 967

✓ 50 sociétés sélectionnées aléatoirement

SOCIÉTÉS SÉLECTIONNÉES

Nombre de sociétés: 50

Statistiques de couverture:
  - Observations moyennes par société: 193
  - Min: 6
  - Max: 252

LISTE DES SOCIÉTÉS
    permno                            comnam ticker  num_observations  first_date   last_date
0    11403        CADENCE DESIGN SYSTEMS INC   CDNS               252  2003-01-31  2023-12-29
1    39917                   WEYERHAEUSER CO     WY               252  2003-01-31  2023-12-29
2    80711    APARTMENT INVESTMENT & MGMT CO    AIV               252  2003-01-31  2023-12-29
3    77661                    D R HORTON INC    DHI               252  2003-01-31  2023-12-29
4    65947                     WELLTOWER INC   WELL               252  2003-01-31  2023-12-29
5    66157                   U S BANCORP DEL    USB               252  2003-01-31  2023-12-29


In [6]:
# Étape 3: Extraire les prix journaliers des actions depuis CRSP
# Utilisation de la table crsp.dsf (Daily Stock File)

query_prices_d = """
SELECT 
    a.permno,
    a.date,
    a.prc,
    a.ret,
    a.shrout,
    a.cfacpr,
    a.cfacshr,
    ABS(a.prc) as price,
    ABS(a.prc) * a.shrout as market_cap
FROM crsp.dsf as a
WHERE a.permno IN ({permnos})
  AND a.date >= '{start_date}'
  AND a.date <= '{end_date}'
ORDER BY a.permno, a.date
""".format(permnos=permnos_str, start_date=start_date, end_date=end_date)

print("Extraction des prix journaliers depuis CRSP...")
daily_prices = db.raw_sql(query_prices_d)

# Convertir la date en datetime
daily_prices['date'] = pd.to_datetime(daily_prices['date'])

print(f"Nombre total d'observations: {len(daily_prices):,}")
print(f"\nPériode couverte: {daily_prices['date'].min()} à {daily_prices['date'].max()}")
print(f"Nombre de jours de trading: {daily_prices['date'].nunique():,}")
print(f"\nPremières lignes:")
daily_prices

Extraction des prix journaliers depuis CRSP...
Nombre total d'observations: 202,300

Période couverte: 2003-01-02 00:00:00 à 2023-12-29 00:00:00
Nombre de jours de trading: 5,285

Premières lignes:


,permno,date,prc,ret,shrout,cfacpr,cfacshr,price,market_cap
0,11403,2003-01-02,11.66,-0.011026,268880.0,1.0,1.0,11.66,3135140.8
1,11403,2003-01-03,9.24,-0.207547,268880.0,1.0,1.0,9.24,2484451.2
2,11403,2003-01-06,9.8,0.060606,268880.0,1.0,1.0,9.8,2635024.0
3,11403,2003-01-07,9.36,-0.044898,268880.0,1.0,1.0,9.36,2516716.8
4,11403,2003-01-08,9.65,0.030983,268880.0,1.0,1.0,9.65,2594692.0
...,...,...,...,...,...,...,...,...,...
202295,93159,2020-08-11,0.4071,-0.027937,199430.0,1.0,1.0,0.4071,81187.953
202296,93159,2020-08-12,0.37,-0.091132,199430.0,1.0,1.0,0.37,73789.1
202297,93159,2020-08-13,0.3731,0.008378,199430.0,1.0,1.0,0.3731,74407.333
202298,93159,2020-08-14,0.33,-0.115519,199430.0,1.0,1.0,0.33,65811.9


In [7]:
# Étape 3.2: Extraire les prix journaliers des actions depuis CRSP
# Utilisation de la table crsp.dsf (Daily Stock File)

query_prices_m = """
SELECT 
    a.permno,
    a.date,
    a.prc,
    a.ret,
    a.shrout,
    a.cfacpr,
    a.cfacshr,
    ABS(a.prc) as price,
    ABS(a.prc) * a.shrout as market_cap
FROM crsp.msf as a
WHERE a.permno IN ({permnos})
  AND a.date >= '{start_date}'
  AND a.date <= '{end_date}'
ORDER BY a.permno, a.date
""".format(permnos=permnos_str, start_date=start_date, end_date=end_date)

print("Extraction des prix journaliers depuis CRSP...")
monthly_prices = db.raw_sql(query_prices_m)

# Convertir la date en datetime
daily_prices['date'] = pd.to_datetime(daily_prices['date'])

print(f"Nombre total d'observations: {len(daily_prices):,}")
print(f"\nPériode couverte: {daily_prices['date'].min()} à {daily_prices['date'].max()}")
print(f"Nombre de jours de trading: {daily_prices['date'].nunique():,}")
print(f"\nPremières lignes:")

monthly_prices

Extraction des prix journaliers depuis CRSP...
Nombre total d'observations: 202,300

Période couverte: 2003-01-02 00:00:00 à 2023-12-29 00:00:00
Nombre de jours de trading: 5,285

Premières lignes:


,permno,date,prc,ret,shrout,cfacpr,cfacshr,price,market_cap
0,11403,2003-01-31,9.92,-0.158609,268880.0,1.0,1.0,9.92,2667289.6
1,11403,2003-02-28,10.59,0.06754,268880.0,1.0,1.0,10.59,2847439.2
2,11403,2003-03-31,10.0,-0.055713,269059.0,1.0,1.0,10.0,2690590.0
3,11403,2003-04-30,11.43,0.143,267434.0,1.0,1.0,11.43,3056770.62
4,11403,2003-05-30,13.9,0.216098,267106.0,1.0,1.0,13.9,3712773.4
...,...,...,...,...,...,...,...,...,...
9663,93159,2020-04-30,0.4555,0.011997,198420.0,1.0,1.0,0.4555,90380.31
9664,93159,2020-05-29,0.3349,-0.264764,205941.0,1.0,1.0,0.3349,68969.6409
9665,93159,2020-06-30,0.6519,0.946551,205941.0,1.0,1.0,0.6519,134252.9379
9666,93159,2020-07-31,0.3921,-0.398527,199430.0,1.0,1.0,0.3921,78196.503


In [8]:
# Étape 4: Préparer les tickers pour IBES
# Utiliser directement les tickers CRSP de nos 50 sociétés sélectionnées

print("="*70)
print("ÉTAPE 4: PRÉPARATION DES TICKERS POUR IBES")
print("="*70)

# Créer un lien PERMNO-TICKER à partir de selected_companies
permno_ticker_link = selected_companies[['permno', 'comnam', 'ticker']].copy()

# Obtenir les tickers uniques (filtrer les valeurs nulles)
tickers = permno_ticker_link[permno_ticker_link['ticker'].notna()]['ticker'].unique().tolist()
tickers_str = "','".join(tickers)

print(f"\nNombre de sociétés: {len(selected_companies)}")
print(f"Nombre de tickers CRSP uniques (non-null): {len(tickers)}")

print(f"\n{'='*70}")
print("TICKERS À UTILISER POUR LES REQUÊTES IBES")
print(f"{'='*70}")
print(sorted(tickers))

print(f"\n{'='*70}")
print("CORRESPONDANCES PERMNO-TICKER")
print(f"{'='*70}")
print(permno_ticker_link[['permno', 'comnam', 'ticker']].to_string())




ÉTAPE 4: PRÉPARATION DES TICKERS POUR IBES

Nombre de sociétés: 50
Nombre de tickers CRSP uniques (non-null): 50

TICKERS À UTILISER POUR LES REQUÊTES IBES
['ACN', 'AIV', 'ALL', 'APD', 'ARG', 'AVP', 'CDNS', 'CL', 'CMG', 'CTRA', 'DHI', 'DNB', 'DNR', 'FAST', 'FII', 'FIS', 'FLR', 'GAS', 'HAR', 'KLAC', 'KORS', 'KRFT', 'KSU', 'LYV', 'MA', 'MERQ', 'MI', 'MIR', 'NDAQ', 'NFB', 'PCG', 'PCP', 'S', 'SANM', 'SCG', 'SEE', 'STI', 'SWY', 'TAP', 'UNH', 'USB', 'VAL', 'WELL', 'WIN', 'WMI', 'WTW', 'WU', 'WY', 'WYE', 'XYL']

CORRESPONDANCES PERMNO-TICKER
    permno                            comnam ticker
0    11403        CADENCE DESIGN SYSTEMS INC   CDNS
1    39917                   WEYERHAEUSER CO     WY
2    80711    APARTMENT INVESTMENT & MGMT CO    AIV
3    77661                    D R HORTON INC    DHI
4    65947                     WELLTOWER INC   WELL
5    66157                   U S BANCORP DEL    USB
6    59248           MOLSON COORS BREWING CO    TAP
7    85926               SEALED AIR CORP NE

In [9]:
# Étape 5: Extraire les données EPS actuelles (Actual EPS) depuis IBES
# Utilisation de la table ibes.act_epsus (Actual EPS)

print("="*70)
print("ÉTAPE 5: EXTRACTION DES EPS DEPUIS IBES")
print("="*70)

query_actual_eps = """
SELECT 
    ticker,
    oftic,
    pends,
    anndats,
    value as actual_eps,
    curr_act as currency,
    pdicity
FROM ibes.act_epsus
WHERE oftic IN ('{tickers}')
  AND anndats >= '{start_date}'
  AND anndats <= '{end_date}'
  AND pdicity = 'ANN'
  AND value IS NOT NULL
ORDER BY oftic, pends
""".format(tickers=tickers_str, start_date=start_date, end_date=end_date)

print(f"Période recherchée: {start_date} à {end_date}")
print(f"Nombre de tickers CRSP à chercher: {len(tickers)}")
print(f"\nExtraction des EPS actuels depuis IBES...")

actual_eps = db.raw_sql(query_actual_eps)

# Convertir les dates
actual_eps['pends'] = pd.to_datetime(actual_eps['pends'])
actual_eps['anndats'] = pd.to_datetime(actual_eps['anndats'])

print(f"\n{'='*70}")
print("RÉSULTATS DE L'EXTRACTION EPS")
print(f"{'='*70}")
print(f"\nNombre total d'observations EPS: {len(actual_eps)}")
print(f"Nombre de tickers IBES (ticker): {actual_eps['ticker'].nunique()}")
print(f"Nombre de tickers CRSP (oftic): {actual_eps['oftic'].nunique()}")

# Afficher le nombre d'EPS par OFTIC (ticker CRSP)
print(f"\n{'='*70}")
print("NOMBRE D'EPS PAR TICKER CRSP (OFTIC)")
print(f"{'='*70}")
eps_count_per_oftic = actual_eps.groupby('oftic').size().sort_values(ascending=False)
print(eps_count_per_oftic)

print(f"\nStatistiques: Min={eps_count_per_oftic.min()}, Max={eps_count_per_oftic.max()}, Moyenne={eps_count_per_oftic.mean():.1f}")

# Afficher aussi par ticker IBES pour voir la correspondance
print(f"\n{'='*70}")
print("CORRESPONDANCE TICKER IBES <-> TICKER CRSP (OFTIC)")
print(f"{'='*70}")
ticker_mapping = actual_eps[['ticker', 'oftic']].drop_duplicates().sort_values('oftic')
print(ticker_mapping.to_string(index=False))

# Identifier les tickers CRSP sans données EPS dans IBES
tickers_set = set(tickers)
tickers_with_eps = set(actual_eps['oftic'].unique())
missing_tickers = tickers_set - tickers_with_eps

if missing_tickers:
    print(f"\n{'='*70}")
    print(f"⚠️  TICKERS CRSP SANS DONNÉES EPS DANS IBES: {len(missing_tickers)}")
    print(f"{'='*70}")
    print(sorted(missing_tickers))
    
    # Trouver les sociétés correspondantes
    missing_companies = permno_ticker_link[permno_ticker_link['ticker'].isin(missing_tickers)]
    if len(missing_companies) > 0:
        print(f"\nSociétés correspondantes:")
        print(missing_companies[['comnam', 'ticker']].to_string(index=False))
        print(f"\n💡 Note: Les tickers CRSP peuvent différer des tickers IBES")
        print(f"   Exemple: ACN (CRSP) = ACNT (IBES) pour Accenture")
        print(f"   Sans table de liaison, ces sociétés ne seront pas trouvées.")

print(f"\n{'='*70}")
print("APERÇU DES DONNÉES EPS")
print(f"{'='*70}")


print(f"\n{'='*70}")
print("EXPLICATION DES RÉSULTATS")
print(f"{'='*70}")
print(f"• Sociétés sélectionnées: {len(selected_companies)}")
print(f"• Tickers CRSP recherchés: {len(tickers)}")
print(f"• Tickers CRSP trouvés dans IBES (oftic): {actual_eps['oftic'].nunique()}")
print(f"• Tickers IBES uniques correspondants: {actual_eps['ticker'].nunique()}")
print(f"\nNote: Un ticker CRSP (oftic) peut correspondre à plusieurs tickers IBES")
print(f"si la société a changé de ticker au fil du temps dans IBES.")

actual_eps


ÉTAPE 5: EXTRACTION DES EPS DEPUIS IBES
Période recherchée: 2003-01-01 à 2023-12-31
Nombre de tickers CRSP à chercher: 50

Extraction des EPS actuels depuis IBES...

RÉSULTATS DE L'EXTRACTION EPS

Nombre total d'observations EPS: 786
Nombre de tickers IBES (ticker): 62
Nombre de tickers CRSP (oftic): 50

NOMBRE D'EPS PAR TICKER CRSP (OFTIC)
oftic
ACN     21
DHI     21
SEE     21
SANM    21
PCG     21
USB     21
NDAQ    21
FLR     21
FAST    21
KLAC    21
WY      21
CL      21
ALL     21
APD     21
UNH     21
AIV     20
KSU     19
S       19
DNB     19
CDNS    18
DNR     18
TAP     18
WTW     18
FII     18
FIS     17
STI     17
CMG     17
AVP     17
MA      17
VAL     17
LYV     17
WU      17
SCG     16
WIN     15
ARG     14
HAR     14
PCP     13
GAS     13
XYL     12
SWY     12
MI       9
CTRA     8
WMI      7
KORS     7
WYE      7
MIR      6
WELL     5
NFB      4
KRFT     3
MERQ     3
dtype: int64

Statistiques: Min=3, Max=21, Moyenne=15.7

CORRESPONDANCE TICKER IBES <-> TICKER CRSP (

,ticker,oftic,pends,anndats,actual_eps,currency,pdicity
0,ACNT,ACN,2003-08-31,2003-10-09,1.05,USD,ANN
1,ACNT,ACN,2004-08-31,2004-10-13,1.23,USD,ANN
2,ACNT,ACN,2005-08-31,2005-10-06,1.46,USD,ANN
3,ACNT,ACN,2006-08-31,2006-09-28,1.61,USD,ANN
4,ACNT,ACN,2007-08-31,2007-09-27,1.97,USD,ANN
...,...,...,...,...,...,...,...
781,XYL,XYL,2018-12-31,2019-01-31,2.88,USD,ANN
782,XYL,XYL,2019-12-31,2020-02-06,3.02,USD,ANN
783,XYL,XYL,2020-12-31,2021-02-04,2.06,USD,ANN
784,XYL,XYL,2021-12-31,2022-02-03,2.49,USD,ANN


### Étape 6: Calcul des ratios P/E glissants (Trailing P/E)

Selon l'article, plusieurs versions du P/E glissant sont calculées:
- **Trailing12mPE** (TTM): Prix(d-1) / Somme des 4 derniers EPS trimestriels
- **PrevFYearPE**: Prix en fin d'année fiscale / EPS annuel réalisé
- **3YearAvgPE**: Moyenne des P/E des 3 dernières années fiscales
- **5YearAvgPE**: Moyenne des P/E des 5 dernières années fiscales

Nous devons extraire les **EPS trimestriels** (pdicity='QTR') pour calculer le TTM PE.

In [10]:
# Étape 5.2: Extraire les données EPS trimestrielles depuis IBES
# Nécessaire pour calculer le Trailing 12-Month PE (somme des 4 derniers trimestres)

print("="*70)
print("ÉTAPE 5.2: EXTRACTION DES EPS TRIMESTRIELS DEPUIS IBES")
print("="*70)

query_quarterly_eps = """
SELECT 
    ticker,
    oftic,
    pends,
    anndats,
    value as quarterly_eps,
    curr_act as currency,
    pdicity
FROM ibes.act_epsus
WHERE oftic IN ('{tickers}')
  AND anndats >= '{start_date}'
  AND anndats <= '{end_date}'
  AND pdicity = 'QTR'
  AND value IS NOT NULL
ORDER BY oftic, pends
""".format(tickers=tickers_str, start_date=start_date, end_date=end_date)

print(f"Période recherchée: {start_date} à {end_date}")
print(f"Nombre de tickers CRSP à chercher: {len(tickers)}")
print(f"\nExtraction des EPS trimestriels depuis IBES...")

quarterly_eps = db.raw_sql(query_quarterly_eps)

# Convertir les dates
quarterly_eps['pends'] = pd.to_datetime(quarterly_eps['pends'])
quarterly_eps['anndats'] = pd.to_datetime(quarterly_eps['anndats'])

print(f"\n{'='*70}")
print("RÉSULTATS DE L'EXTRACTION EPS TRIMESTRIELS")
print(f"{'='*70}")
print(f"\nNombre total d'observations EPS trimestriels: {len(quarterly_eps)}")
print(f"Nombre de tickers CRSP (oftic): {quarterly_eps['oftic'].nunique()}")

# Afficher le nombre d'EPS trimestriels par ticker
print(f"\n{'='*70}")
print("NOMBRE D'EPS TRIMESTRIELS PAR TICKER CRSP (OFTIC)")
print(f"{'='*70}")
qtr_eps_count = quarterly_eps.groupby('oftic').size().sort_values(ascending=False)
print(qtr_eps_count.head(20))
print(f"\nStatistiques: Min={qtr_eps_count.min()}, Max={qtr_eps_count.max()}, Moyenne={qtr_eps_count.mean():.1f}")

print(f"\n✓ EPS trimestriels extraits avec succès!")
print(f"  → Variable: quarterly_eps")

quarterly_eps.head(20)

ÉTAPE 5.2: EXTRACTION DES EPS TRIMESTRIELS DEPUIS IBES
Période recherchée: 2003-01-01 à 2023-12-31
Nombre de tickers CRSP à chercher: 50

Extraction des EPS trimestriels depuis IBES...

RÉSULTATS DE L'EXTRACTION EPS TRIMESTRIELS

Nombre total d'observations EPS trimestriels: 3154
Nombre de tickers CRSP (oftic): 50

NOMBRE D'EPS TRIMESTRIELS PAR TICKER CRSP (OFTIC)
oftic
ACN     85
DHI     84
SEE     84
SANM    84
PCG     84
USB     84
NDAQ    84
FLR     84
FAST    84
KLAC    84
WY      84
CL      84
ALL     84
APD     84
UNH     84
AIV     78
DNB     77
KSU     76
TAP     75
S       75
dtype: int64

Statistiques: Min=10, Max=85, Moyenne=63.1

✓ EPS trimestriels extraits avec succès!
  → Variable: quarterly_eps


,ticker,oftic,pends,anndats,quarterly_eps,currency,pdicity
0,ACNT,ACN,2002-11-30,2003-01-09,0.27,USD,QTR
1,ACNT,ACN,2003-02-28,2003-04-14,0.25,USD,QTR
2,ACNT,ACN,2003-05-31,2003-07-15,0.28,USD,QTR
3,ACNT,ACN,2003-08-31,2003-10-09,0.25,USD,QTR
4,ACNT,ACN,2003-11-30,2004-01-13,0.27,USD,QTR
5,ACNT,ACN,2004-02-29,2004-03-30,0.29,USD,QTR
6,ACNT,ACN,2004-05-31,2004-07-07,0.37,USD,QTR
7,ACNT,ACN,2004-08-31,2004-10-13,0.3,USD,QTR
8,ACNT,ACN,2004-11-30,2005-01-06,0.32,USD,QTR
9,ACNT,ACN,2005-02-28,2005-04-07,0.32,USD,QTR


In [11]:
print("="*70)
print("ÉTAPE 6: CALCUL DES RATIOS P/E GLISSANTS (TTM PE)")
print("="*70)
print("Selon l'article: TTM PE = Prix(d-1) / Somme des 4 derniers EPS trimestriels")
print("où les 4 derniers EPS sont basés sur anndats (date d'annonce)")

# 1. Fusionner daily_prices avec permno_ticker_link pour obtenir les tickers
print("\n1. Ajout des tickers CRSP aux prix journaliers...")
daily_prices_with_ticker = daily_prices.merge(
    permno_ticker_link[['permno', 'ticker']], 
    on='permno', 
    how='left'
)

print(f"   Prix journaliers: {len(daily_prices_with_ticker):,} observations")
print(f"   Tickers uniques: {daily_prices_with_ticker['ticker'].nunique()}")
print(f"   Période: {daily_prices_with_ticker['date'].min()} à {daily_prices_with_ticker['date'].max()}")

# 2. Préparer les données trimestrielles
print("\n2. Préparation des EPS trimestriels...")
quarterly_eps_work = quarterly_eps[['oftic', 'pends', 'anndats', 'quarterly_eps']].copy()
quarterly_eps_work = quarterly_eps_work.rename(columns={'oftic': 'ticker'})

print(f"   EPS trimestriels: {len(quarterly_eps_work):,} observations")
print(f"   Tickers avec EPS: {quarterly_eps_work['ticker'].nunique()}")

ÉTAPE 6: CALCUL DES RATIOS P/E GLISSANTS (TTM PE)
Selon l'article: TTM PE = Prix(d-1) / Somme des 4 derniers EPS trimestriels
où les 4 derniers EPS sont basés sur anndats (date d'annonce)

1. Ajout des tickers CRSP aux prix journaliers...
   Prix journaliers: 202,300 observations
   Tickers uniques: 50
   Période: 2003-01-02 00:00:00 à 2023-12-29 00:00:00

2. Préparation des EPS trimestriels...
   EPS trimestriels: 3,154 observations
   Tickers avec EPS: 50


In [12]:


# 3. Calculer le Trailing 12-Month PE (TTM PE)
# Pour chaque jour de trading d, on utilise le prix de d-1 et les 4 derniers EPS annoncés avant d
print("\n3. Calcul du TTM PE pour chaque jour de trading...")
print("   (Peut prendre quelques minutes pour ~130K observations journalières)")

# Créer le prix de d-1 pour chaque ligne
# Trier par ticker et date
daily_sorted = daily_prices_with_ticker.sort_values(['ticker', 'date']).copy()

# Créer le prix du jour précédent (lag de 1 jour)
daily_sorted['price_d_minus_1'] = daily_sorted.groupby('ticker')['price'].shift(1)

# Garder seulement les lignes avec un prix d-1 disponible
daily_with_lag = daily_sorted[daily_sorted['price_d_minus_1'].notna()].copy()

print(f"   Observations avec prix d-1: {len(daily_with_lag):,}")

# Fonction optimisée pour calculer le TTM EPS
def calculate_ttm_eps_daily(price_data, eps_data):
    """
    Calculer le TTM EPS pour chaque jour de trading.
    TTM EPS = somme des 4 derniers EPS trimestriels annoncés avant la date d.
    """
    results = []
    
    # Grouper par ticker pour efficacité
    for ticker in price_data['ticker'].unique():
        if pd.isna(ticker):
            continue
        
        ticker_prices = price_data[price_data['ticker'] == ticker].sort_values('date')
        ticker_eps = eps_data[eps_data['ticker'] == ticker].sort_values('anndats')
        
        if len(ticker_eps) < 4:
            # Pas assez d'EPS pour calculer TTM
            continue
        
        for _, price_row in ticker_prices.iterrows():
            trading_date = price_row['date']
            
            # Trouver tous les EPS annoncés AVANT ou À cette date de trading
            available_eps = ticker_eps[ticker_eps['anndats'] <= trading_date]
            
            if len(available_eps) >= 4:
                # Prendre les 4 derniers EPS (les plus récents basés sur anndats)
                last_4_eps = available_eps.sort_values('anndats', ascending=False).head(4)
                ttm_eps = last_4_eps['quarterly_eps'].sum()
                
                # Calculer le TTM PE
                ttm_pe = price_row['price_d_minus_1'] / ttm_eps if ttm_eps > 0 else np.nan
                
                result = {
                    'permno': price_row['permno'],
                    'ticker': ticker,
                    'date': trading_date,
                    'price_d': price_row['price'],
                    'price_d_minus_1': price_row['price_d_minus_1'],
                    'ttm_eps': ttm_eps,
                    'TTM_PE': ttm_pe,
                    'num_quarters': 4,
                    'latest_eps_anndats': last_4_eps['anndats'].max(),
                    'oldest_eps_anndats': last_4_eps['anndats'].min()
                }
                results.append(result)
    
    return pd.DataFrame(results)

# Calculer le TTM PE pour tous les tickers
print("   Calcul en cours...")
ttm_pe_daily = calculate_ttm_eps_daily(daily_with_lag, quarterly_eps_work)

print(f"\n{'='*70}")
print("RÉSULTATS TTM PE (DAILY)")
print(f"{'='*70}")
print(f"\nObservations totales avec TTM PE calculé: {len(ttm_pe_daily):,}")
print(f"Tickers couverts: {ttm_pe_daily['ticker'].nunique()}")
print(f"Période couverte: {ttm_pe_daily['date'].min()} à {ttm_pe_daily['date'].max()}")

# Filtrer les EPS positifs pour statistiques
ttm_pe_clean = ttm_pe_daily[
    (ttm_pe_daily['ttm_eps'] > 0) & 
    (ttm_pe_daily['TTM_PE'] > 0) &
    (ttm_pe_daily['TTM_PE'].notna())
].copy()

print(f"\nObservations avec TTM EPS positif et TTM PE valide: {len(ttm_pe_clean):,}")
print(f"Tickers avec données valides: {ttm_pe_clean['ticker'].nunique()}")

print(f"\n{'='*70}")
print("STATISTIQUES DES TTM PE (EPS positifs)")
print(f"{'='*70}")
print(ttm_pe_clean['TTM_PE'].describe())

print(f"\n{'='*70}")
print("COUVERTURE PAR SOCIÉTÉ (TOP 10)")
print(f"{'='*70}")
coverage = ttm_pe_clean.groupby('ticker').agg({
    'date': ['min', 'max', 'count'],
    'TTM_PE': ['mean', 'median', 'std']
}).round(2)
coverage.columns = ['Date_debut', 'Date_fin', 'N_obs', 'TTM_PE_mean', 'TTM_PE_median', 'TTM_PE_std']
coverage = coverage.sort_values('N_obs', ascending=False)
print(coverage.head(10))

print(f"\n{'='*70}")
print("APERÇU DES DONNÉES (premières lignes)")
print(f"{'='*70}")
print(ttm_pe_clean[['ticker', 'date', 'price_d_minus_1', 'ttm_eps', 'TTM_PE', 'latest_eps_anndats']].head(20))

print(f"\n✓ Calcul TTM PE terminé avec succès!")
print(f"  → Variable: ttm_pe_daily (toutes observations)")
print(f"  → Variable: ttm_pe_clean (EPS positifs seulement)")



3. Calcul du TTM PE pour chaque jour de trading...
   (Peut prendre quelques minutes pour ~130K observations journalières)
   Observations avec prix d-1: 202,250
   Calcul en cours...

RÉSULTATS TTM PE (DAILY)

Observations totales avec TTM PE calculé: 186,121
Tickers couverts: 49
Période couverte: 2003-10-08 00:00:00 à 2023-12-29 00:00:00

Observations avec TTM EPS positif et TTM PE valide: 171,233
Tickers avec données valides: 49

STATISTIQUES DES TTM PE (EPS positifs)
count    1.712330e+05
mean     2.592107e+14
std      1.099448e+16
min      2.564103e-01
25%      1.316667e+01
50%      1.847768e+01
75%      2.957457e+01
max      6.234783e+17
Name: TTM_PE, dtype: float64

COUVERTURE PAR SOCIÉTÉ (TOP 10)
       Date_debut   Date_fin  N_obs  TTM_PE_mean  TTM_PE_median  TTM_PE_std
ticker                                                                     
ACN    2003-10-09 2023-12-29   5091        21.58          20.81        5.85
FAST   2003-10-10 2023-12-29   5090       188.51         

,permno,comnam,ticker,num_observations,first_date,last_date
0,11403,CADENCE DESIGN SYSTEMS INC,CDNS,252,2003-01-31,2023-12-29
1,39917,WEYERHAEUSER CO,WY,252,2003-01-31,2023-12-29
2,80711,APARTMENT INVESTMENT & MGMT CO,AIV,252,2003-01-31,2023-12-29
3,77661,D R HORTON INC,DHI,252,2003-01-31,2023-12-29
4,65947,WELLTOWER INC,WELL,252,2003-01-31,2023-12-29
5,66157,U S BANCORP DEL,USB,252,2003-01-31,2023-12-29
6,59248,MOLSON COORS BREWING CO,TAP,252,2003-01-31,2023-12-29
7,85926,SEALED AIR CORP NEW,SEE,252,2003-01-31,2023-12-29
8,46886,K L A TENCOR CORP,KLAC,252,2003-01-31,2023-12-29
9,76082,COTERRA ENERGY INC,CTRA,252,2003-01-31,2023-12-29


In [ ]:
daily_prices_with_ticker



,permno,date,prc,ret,shrout,cfacpr,cfacshr,price,market_cap,ticker
0,11403,2003-01-02,11.66,-0.011026,268880.0,1.0,1.0,11.66,3135140.8,CDNS
1,11403,2003-01-03,9.24,-0.207547,268880.0,1.0,1.0,9.24,2484451.2,CDNS
2,11403,2003-01-06,9.8,0.060606,268880.0,1.0,1.0,9.8,2635024.0,CDNS
3,11403,2003-01-07,9.36,-0.044898,268880.0,1.0,1.0,9.36,2516716.8,CDNS
4,11403,2003-01-08,9.65,0.030983,268880.0,1.0,1.0,9.65,2594692.0,CDNS
...,...,...,...,...,...,...,...,...,...,...
202295,93159,2020-08-11,0.4071,-0.027937,199430.0,1.0,1.0,0.4071,81187.953,VAL
202296,93159,2020-08-12,0.37,-0.091132,199430.0,1.0,1.0,0.37,73789.1,VAL
202297,93159,2020-08-13,0.3731,0.008378,199430.0,1.0,1.0,0.3731,74407.333,VAL
202298,93159,2020-08-14,0.33,-0.115519,199430.0,1.0,1.0,0.33,65811.9,VAL


In [17]:
# Afficher les données pour DNB
dnb_data = daily_prices_with_ticker[daily_prices_with_ticker['ticker'] == 'S'].copy()

if len(dnb_data) > 0:
    print(f"{'='*70}")
    print(f"{'='*70}")
    print(f"\nNombre d'observations: {len(dnb_data)}")
    print(f"Première date: {dnb_data['date'].min()}")
    print(f"Dernière date: {dnb_data['date'].max()}")
    print(f"PERMNO: {dnb_data['permno'].iloc[0]}")
    
    print(f"\n{'='*70}")
    print("APERÇU DES DONNÉES")
    print(f"{'='*70}")
    print("\nPremières lignes:")
    print(dnb_data[['permno', 'ticker', 'date', 'price', 'ret']].head(10))
    
    print("\nDernières lignes:")
    print(dnb_data[['permno', 'ticker', 'date', 'price', 'ret']].tail(10))
else:
    print("⚠️ Aucune donnée trouvée pour le ticker S")
    print("\nTickers disponibles:")
    print(sorted(daily_prices_with_ticker['ticker'].unique()))


Nombre d'observations: 561
Première date: 2003-01-02 00:00:00
Dernière date: 2005-03-24 00:00:00
PERMNO: 14322

APERÇU DES DONNÉES

Premières lignes:
       permno ticker       date  price       ret
32691   14322      S 2003-01-02  24.58  0.026305
32692   14322      S 2003-01-03  23.99 -0.024003
32693   14322      S 2003-01-06   24.6  0.025427
32694   14322      S 2003-01-07   24.7  0.004065
32695   14322      S 2003-01-08  25.05   0.01417
32696   14322      S 2003-01-09  27.29  0.089421
32697   14322      S 2003-01-10  27.59  0.010993
32698   14322      S 2003-01-13  27.51   -0.0029
32699   14322      S 2003-01-14  27.51       0.0
32700   14322      S 2003-01-15   26.7 -0.029444

Dernières lignes:
       permno ticker       date  price       ret
33242   14322      S 2005-03-11  57.56  0.078711
33243   14322      S 2005-03-14   58.0  0.007644
33244   14322      S 2005-03-15  58.45  0.007759
33245   14322      S 2005-03-16  57.21 -0.021215
33246   14322      S 2005-03-17  56.88 -0.0057